In [1]:
import os
from IPython.display import Audio
import numpy as np
import math
import copy
from shutil import copyfile
import glob
import pytsmod as tsm
import soundfile as sf 

In [ ]:
np.random.seed(42)

In [ ]:
def moveFiles():
    path = "/mnt/data0/Projects/PianoTrioAlign/URMP/Dataset/test"
    pieces = os.listdir(path)

    for piece in pieces:
        dirpath = os.path.join(path, piece)
        audio_files = []
        for file in glob.glob(f'{dirpath}/AuSep*.wav'):
            audio_files.append(file)
        audio_files = sorted(audio_files)
        fullmix_audio = glob.glob(f'{dirpath}/AuMix*.wav')[0]
        out_dir = '/mnt/data0/Projects/PianoTrioAlign/PianoTrioAlignment/URMP-clean/data/test'
        out_piece_dir = os.path.join(out_dir, piece)
        os.makedirs(out_piece_dir,exist_ok=True)
        for idx, file in enumerate(audio_files):
            copyfile(file, os.path.join(out_piece_dir, f'I{idx}.wav'))
        copyfile(fullmix_audio, os.path.join(out_piece_dir, f'Mix{idx}.wav'))
        
        annot_dir =  os.path.join(dirpath, 'annot1')
        fullmix_annot = os.path.join(annot_dir, 'aumix.txt')
        annot_files = []
        for file in glob.glob(f'{annot_dir}/Notes_*.txt'):
            annot_files.append(file)
        annot_files = sorted(annot_files)
        for idx, file in enumerate(annot_files):
            copyfile(file, os.path.join(out_piece_dir, f'I{idx}.txt'))
        copyfile(fullmix_annot, os.path.join(out_piece_dir, f'Mix{idx}.txt'))
moveFiles()

In [ ]:
def interpolate(data, s_ap, sr):
    s_ap = s_ap / sr
    new_times = []
    for time, idx in data:
        time = float(time)
        for i in range(3):
            if s_ap[0][i+1] > time:
                elapsed = time-s_ap[0][i]
                slope = (s_ap[1][i+1]-s_ap[1][i])/(s_ap[0][i+1]-s_ap[0][i])
                new_time = elapsed*slope+s_ap[1][i]
                new_times.append((new_time, idx))
                break
    return new_times


def timeWarpFile():
    path = "/mnt/data0/Projects/PianoTrioAlign/PianoTrioAlignment/URMP-clean/data/test"
    pieces = os.listdir(path)
    for piece in pieces:
        print(piece)
        dirpath = os.path.join(path, piece)
        fullmix_annot = glob.glob(dirpath+f'/Mix*.txt')[0]
        fullmix_file = glob.glob(dirpath+f'/Mix*.wav')[0]
        data = []
        with open(fullmix_annot,'r') as f:
            for line in f:
                data.append(line.strip().split())
        audio, sr = sf.read(fullmix_file)
        
        num_samples = 10
        for i in range(num_samples):
            tempos = np.exp(np.random.uniform(math.log(0.8),math.log(1.2), 3))
            factors = 1/tempos
            L = len(audio)
            audio = audio.T
            s_ap = np.array([[0, L / 3, 2 * L / 3, L], [0, L/3*factors[0], L/3*factors[0]+L/3*factors[1], L/3*factors[0]+L/3*factors[1]+L/3*factors[2]]])
            audio_stretched = tsm.phase_vocoder(audio, s_ap)
            sf.write(os.path.join(dirpath, f'Vocoder{i}.wav'), audio_stretched, sr)
        
            
            new_times = interpolate(data, s_ap, sr)
            pv_out = open(os.path.join(dirpath, f'Vocoder{i}.txt'), 'w')

            for time, idx in new_times:
                print(time, idx, file=pv_out)

            pv_out.close()

timeWarpFile()